In [ ]:
# --- Standard Library ---
import os
import sys
import json

# --- Google Cloud Auth + APIs ---
from google.cloud import bigquery
from google.oauth2 import service_account
from google.api_core.exceptions import GoogleAPICallError, RetryError

from gspread_dataframe import set_with_dataframe

# --- Data & Visualization ---
import pandas as pd
import openpyxl

# --- Local Modules ---
from modules.pull_and_backup import pull_and_append
from modules.flattening import flatten_extract_params, flatten_row, flatten_nested_column
from modules.lists_and_maps import df_column_order, mini_game_related_column_order, mini_game_related_value_map

In [2]:
# --- Configuration ---
SERVICE_ACCOUNT_KEY = './keys/key.json'
DATA_PATH = './data/data.json'
PROJECT_ID = "emojioracle-342f1"
DATASET_ID = "analytics_481352676"
BACKUP_PATH = './backup/'
SCOPES = [
    "https://www.googleapis.com/auth/bigquery",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
# --- BigQuery Client ---
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY,
    scopes = SCOPES
)
bq_client = bigquery.Client(credentials = credentials, project = PROJECT_ID)

In [3]:
# will pull what is missing from BigQuery and merge into data_path
df = pd.DataFrame(pull_and_append(credentials = credentials, project_id = PROJECT_ID, dataset_id = DATASET_ID, data_path = DATA_PATH, backup_path = BACKUP_PATH))
print(f"Data loaded with {len(df)} rows and {len(df.columns)} columns.")

Loaded existing data.
Latest event_date in merged data: 20250605
Backup already exists: events_20250407
Backup already exists: events_20250408
Backup already exists: events_20250409
Backup already exists: events_20250410
Backup already exists: events_20250411
Backup already exists: events_20250412
Backup already exists: events_20250413
Backup already exists: events_20250414
Backup already exists: events_20250415
Backup already exists: events_20250416
Backup already exists: events_20250417
Backup already exists: events_20250418
Backup already exists: events_20250420
Backup already exists: events_20250421
Backup already exists: events_20250422
Backup already exists: events_20250423
Backup already exists: events_20250424
Backup already exists: events_20250425
Backup already exists: events_20250427
Backup already exists: events_20250428
Backup already exists: events_20250429
Backup already exists: events_20250430
Backup already exists: events_20250506
Backup already exists: events_20250507

In [4]:
# load the data from the JSON file
df = pd.read_json(DATA_PATH)

In [5]:
pd.set_option('display.max_columns', None) # uncomment to see all of the cols in pandas dataframes

In [6]:
# flattenning

df = pd.DataFrame([flatten_row(row) for _, row in df.iterrows()]) # for wtfs refer to ./modules/flattening_json.py

In [7]:
# cleaning & preprocessing

## dates and times
df = df.drop(columns=['event_date']) # built in case event_date may not be the same as the one in the event_timestamp

# convert unix time to ape-friendly datetime format
df['time_delta'] = pd.to_datetime(df['event_timestamp'], unit='us', utc=True) - pd.to_datetime(df['event_previous_timestamp'], unit='us', utc=True)
df['time_delta'] = df['time_delta'].dt.total_seconds() # convert to seconds

df['event_datetime'] = pd.to_datetime(df['event_timestamp'], unit='us', utc=True) 
df['event_previous_datetime'] = pd.to_datetime(df['event_previous_timestamp'], unit='us', utc=True)
df['event_first_touch_datetime'] = pd.to_datetime(df['user_first_touch_timestamp'], unit='us', utc=True)
df['user.first_open_datetime'] = pd.to_datetime(df['user.first_open_time'], unit='ms', utc=True)


df['event_date'] = df['event_datetime'].dt.normalize()
df['event_time'] = df['event_datetime'].dt.time

df['event_previous_date'] = df['event_previous_datetime'].dt.normalize()
df['event_previous_time'] = df['event_previous_datetime'].dt.time

df['event_first_touch_date'] = df['event_first_touch_datetime'].dt.normalize()
df['event_first_touch_time'] = df['event_first_touch_datetime'].dt.time

df['user.first_open_date'] = df['user.first_open_datetime'].dt.normalize()
df['user.first_open_time'] = df['user.first_open_datetime'].dt.time

df['device.time_zone_offset_hours'] = df['device.time_zone_offset_seconds'] / 3600 # seconds to hours
df['event_params.engagement_time_seconds'] = df['event_params.engagement_time_msec'] / 1000 # ms to seconds
df['event_server_delay_seconds'] = df['event_server_timestamp_offset'] / 1000 # ms to seconds 
df['event_params.time_spent_seconds'] = df['event_params.time_spent'] # just renaming for clarity

df = df.drop(columns=['event_timestamp',
                      'event_previous_timestamp', 
                      'user_first_touch_timestamp', 
                      'event_server_timestamp_offset', 
                      'device.time_zone_offset_seconds', 
                      'event_params.engagement_time_msec',
                      'event_previous_datetime',
                      'event_params.time_spent',
                      'event_first_touch_datetime',
                      'user.first_open_datetime'
                    ])



In [8]:
## time series

df['ts_weekday'] = df['event_datetime'].dt.day_name() # weekday name
df['ts_weekday'] = pd.Categorical(df['ts_weekday'], categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True) # order the weekdays

df['ts_local_time'] = df['event_datetime'] + pd.to_timedelta(df['device.time_zone_offset_hours'].fillna(0), unit='h') # local time
df['ts_hour'] = df['ts_local_time'].dt.hour # local hour
df['ts_daytime_named'] = df['ts_hour'].apply(lambda x: 
                                             'night' if (x < 6 or x > 22) else 
                                             'morning' if x < 11 else 
                                             'noon' if x < 14 else 
                                             'afternoon' if x < 17 else 
                                             'evening') # time group of day
df['ts_is_weekend'] = df['ts_weekday'].isin(['Saturday', 'Sunday']) # is weekend

In [9]:
# Revise Question Indices as df['event_params.current_question_index'] 
"""
Tier 1: 16 Questions, Except t: 12
Tier 2: 12 Questions
Tier 3: 12 Questions
Tier 4: 10 Questions
"""

df['event_params.current_question_index'] = pd.Series([pd.NA] * len(df), dtype="Int64")

df['event_params.current_tier'] = pd.to_numeric(df['event_params.current_tier'], errors='coerce').astype("Int64")
df['event_params.current_qi'] = pd.to_numeric(df['event_params.current_qi'], errors='coerce').astype("Int64")

notna_mask = df['event_params.character_name'].notna()

# Tier 1
tier_1_mask = notna_mask & (df['event_params.current_tier'] == 1)
t_char_mask = tier_1_mask & (df['event_params.character_name'] == 't')

df.loc[t_char_mask, 'event_params.current_question_index'] = 13 - df.loc[t_char_mask, 'event_params.current_qi']
df.loc[~t_char_mask & tier_1_mask, 'event_params.current_question_index'] = 17 - df.loc[(~t_char_mask) & tier_1_mask, 'event_params.current_qi']

# Tier 2 & 3
tier_2_3_mask = notna_mask & df['event_params.current_tier'].isin([2, 3])
df.loc[tier_2_3_mask, 'event_params.current_question_index'] = 13 - df.loc[tier_2_3_mask, 'event_params.current_qi']

# Tier 4
tier_4_mask = notna_mask & (df['event_params.current_tier'] == 4)
df.loc[tier_4_mask, 'event_params.current_question_index'] = 7 - df.loc[tier_4_mask, 'event_params.current_qi']

# Hiccups
problems_mask = notna_mask & ~df['event_params.current_tier'].isin([1, 2, 3, 4])
if df[problems_mask].shape[0] > 0:
    print("Something wrong in:")
    print(df.loc[problems_mask, ['event_params.character_name', 'event_params.current_tier', 'event_params.current_qi']])


In [10]:
# I reordered the columns to make it easier to follow: list is in columns.py

df = df[df_column_order]

df.columns = df.columns.str.replace('.', '__') 
# replaced dots with underscores in column names bc of bigquerys nagging
# event_params.* columns are replaced with event_params__* (double underscores) 
# to avoid conflicts with the original columns

In [11]:
df.to_parquet('./data/cleaned_data.parquet', index=False)

In [12]:
# Ensure events are sorted per user
df_sorted = df.sort_values(by=['user_pseudo_id', 'event_datetime'])

# Compute time gap between events per user
df_sorted['time_diff'] = df_sorted.groupby('user_pseudo_id')['event_datetime'].diff()

# Use 6-minute timeout
SESSION_TIMEOUT = pd.Timedelta(minutes=6)

# Define inferred session ID using 6-minute gaps
df_sorted['inferred_session_id'] = (
    (df_sorted['time_diff'] > SESSION_TIMEOUT) | df_sorted['time_diff'].isna()
).cumsum()

# Aggregate inferred sessions
session_times = df_sorted.groupby(['user_pseudo_id', 'inferred_session_id'])['event_datetime'].agg(['min', 'max'])
session_times['session_screen_time_seconds'] = (session_times['max'] - session_times['min']).dt.total_seconds()
session_times = session_times[session_times['session_screen_time_seconds'] > 0]
session_times = session_times.reset_index()

session_times = session_times.rename(columns={'min': 'date_time'})
session_times['local_datetime'] = session_times['date_time'] + pd.to_timedelta(df_sorted['device__time_zone_offset_hours'].fillna(0), unit='h')

session_times['local_date'] = session_times['local_datetime'].dt.normalize()
ts_weekday = session_times['local_datetime'].dt.day_name()
session_times['ts_weekday'] = pd.Categorical(ts_weekday, categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
session_times['ts_is_weekend'] = session_times['ts_weekday'].isin(['Saturday', 'Sunday']) # is weekend

session_times['local_time'] = session_times['local_datetime'].dt.time 
session_times['ts_hour'] = session_times['local_datetime'].dt.hour
session_times['ts_daytime_named'] = session_times['ts_hour'].apply(lambda x: 
                                             'night' if (x < 6 or x > 22) else 
                                             'morning' if x < 11 else 
                                             'noon' if x < 14 else 
                                             'afternoon' if x < 17 else 
                                             'evening') # time group of day

session_times = session_times[['inferred_session_id', 'session_screen_time_seconds', 'date_time', 'local_datetime', 'local_date', 'local_time', 'ts_weekday', 'ts_is_weekend', 'ts_hour', 'ts_daytime_named', 'user_pseudo_id']]



In [44]:
mini_game_related = df[
    df['event_name'].isin(['mini_game_started', 'mini_game_completed', 'mini_game_failed']) |
    df['event_params__where_its_earned'].notna() |
    df['event_params__how_its_earned'].str.contains('mini', case=False, na=False)
].copy()


In [45]:
mini_game_related = mini_game_related[list(mini_game_related_column_order.keys())].rename(columns=mini_game_related_column_order)

In [46]:
#split 'event_params_mini_game_ri' maze_hand_* into columns
# e.g 'maze_hand_WomanHandTwo_maze_level_3'

# Column to process
col = 'mini_game_ri'

# Filter rows starting with 'maze_hand'
mask = mini_game_related[col].str.startswith('maze_hand', na=False)

# Split the matching rows by underscore
parts = mini_game_related.loc[mask, col].str.split('_', expand=True)

# Extract Gender and Hand using the updated regex
gender_hand = parts[2].str.extract(r'(?P<Gender>Woman|Man)Hand(?P<Hand>\w+)')

# Extract Level (assumed to be in the last part)
levels = parts[5]

# Create new columns with extracted data
mini_game_related.loc[mask, 'maze_gender'] = gender_hand['Gender']
mini_game_related.loc[mask, 'maze_hand'] = gender_hand['Hand']
mini_game_related.loc[mask, 'maze_level'] = levels

In [47]:
# split event_params_mini_game_ri buff_* into columns
# e.g. 'buff_IncreaseXEnergy_gift_True_gold_False'


# Column to process
col = 'mini_game_ri'

# Filter rows starting with 'buff'
mask = mini_game_related[col].str.startswith('buff', na=False)

# Split the matching rows by underscore
parts = mini_game_related.loc[mask, col].str.split('_', expand=True)

# Extract Buff Type and Level
buff_type = parts[2].str.extract(r'(?P<BuffType>\w+)')

# Extract Buff Gift and Gold status
buff_gift = parts[3].str.extract(r'(?P<BuffGift>\w+)')
buff_gold = parts[5].str.extract(r'(?P<BuffGold>\w+)')

# Create new columns with extracted data
mini_game_related.loc[mask, 'buff_type'] = buff_type['BuffType']
mini_game_related.loc[mask, 'buff_gift'] = buff_gift['BuffGift'].str.lower() == 'true'
mini_game_related.loc[mask, 'buff_gold'] = buff_gold['BuffGold'].str.lower() == 'true'

In [ ]:
# Replace values in mini_game_ri with the mapping from mini_game_related_value_map
# mini_game_related_value_map is a dictionary mapping from the original mini_game_ri values to more readable names

cols_to_replace = ['mini_game_name', 'mini_game_ri', 'where_earned', 'how_earned', 'maze_gender', 'maze_hand', 'maze_level', 'buff_type','buff_gift', 'buff_gold']

for col in cols_to_replace:
    mini_game_related[col] = mini_game_related[col].replace(mini_game_related_value_map)

In [63]:
mini_game_related.head(20)

,event_name,event_date,event_time,mini_game_name,mini_game_ri,time_spent,where_earned,how_earned,amount_earned,maze_gender,maze_hand,maze_level,buff_type,buff_gift,buff_gold
1413,mini_game_started,2025-04-04 00:00:00+00:00,21:19:38.297060,Stone Mini Game,Stone Game,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1414,mini_game_started,2025-04-04 00:00:00+00:00,22:06:08.059051,Cauldron Mini Game,Cauldron Game,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1415,mini_game_started,2025-04-05 00:00:00+00:00,06:11:22.085062,Cauldron Mini Game,Cauldron Game,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1416,mini_game_started,2025-04-05 00:00:00+00:00,06:12:18.848066,Stone Mini Game,Stone Game,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1417,mini_game_started,2025-04-05 00:00:00+00:00,09:38:21.004169,Star Mini Game,Taurus Zodiac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1418,mini_game_started,2025-04-05 00:00:00+00:00,11:56:43.966114,Stone Mini Game,Stone Game,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1419,mini_game_started,2025-04-05 00:00:00+00:00,17:36:39.660075,Stone Mini Game,Stone Game,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1420,mini_game_started,2025-04-05 00:00:00+00:00,17:33:57.975051,Maze Mini Game,maze_hand_WomanHandOne_maze_level_4,NaN,NaN,NaN,NaN,Woman,One,4,NaN,NaN,NaN
1421,mini_game_started,2025-04-05 00:00:00+00:00,17:40:10.672098,Maze Mini Game,maze_hand_ManHandOne_maze_level_6,NaN,NaN,NaN,NaN,Man,One,6,NaN,NaN,NaN
1422,mini_game_started,2025-04-04 00:00:00+00:00,21:16:19.940035,Cauldron Mini Game,Cauldron Game,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
session_times.to_parquet('./data/inferred_sessions.parquet', index=False)

In [ ]:
# Define table names
main_table_id = f"{PROJECT_ID}.{DATASET_ID}.clean_data"
sessions_table_id = f"{PROJECT_ID}.{DATASET_ID}.inferred_sessions"

try:
    # Upload main event data
    main_job = bq_client.load_table_from_dataframe(
        df,
        main_table_id,
        job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    )
    main_job.result()  # Wait for main job to finish

    print(f"Cleaned event data written to {main_table_id}")

    # Upload session summary
    sessions_job = bq_client.load_table_from_dataframe(
        session_times,
        sessions_table_id,
        job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    )
    sessions_job.result()  # Wait for sessions job to finish

    print(f"Session durations written to {sessions_table_id}")
    print("Data processing and upload completed successfully.")
except GoogleAPICallError as api_error:
    print(f"API error occurred: {api_error}")
except RetryError as retry_error:
    print(f"Retry error occurred: {retry_error}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
dftz = df.copy()
for col in df.select_dtypes(include=['datetimetz']):
    df[col] = df[col].dt.tz_localize(None)

session_times_tz = session_times.copy()
for col in session_times.select_dtypes(include=['datetimetz']):
    session_times[col] = session_times[col].dt.tz_localize(None)

df.to_excel('./data/cleaned_data.xlsx', index=False)
session_times.to_excel('./data/inferred_sessions.xlsx', index=False)    

In [ ]:
df.size

AttributeError: 'list' object has no attribute 'astype'